Table 4.10.1: Injecting status of new presentations to treatment

In [1]:
from gssutils import *

if is_interactive():
    scraper = Scraper('https://www.gov.uk/government/collections/alcohol-and-drug-misuse-and-treatment-statistics')
    scraper.select_dataset(title=lambda x: x.startswith('Substance misuse treatment for adults'), latest=True)
    tabs = {tab.name: tab for tab in scraper.distribution(title=lambda x: x.startswith('Data tables')).as_databaker()}

In [2]:
tab = tabs['Table 4.10.1']

https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/752515/AdultSubstanceMisuseNDTMSDataTables2017-18.xlsx

In [3]:
cell = tab.filter('Injecting Status')
cell.assert_one()
observations = tab.filter('n').fill(DOWN).is_not_blank().is_not_whitespace()
InjectingStatus = cell.expand(DOWN).is_not_blank().is_not_whitespace()
Clients = cell.expand(RIGHT).is_not_blank().is_not_whitespace()
Dimensions = [
            HDim(InjectingStatus,'Treatment group',DIRECTLY,LEFT),
            HDim(Clients,'Clients in treatment',CLOSEST,LEFT),
            HDimConst('Measure Type','Count'),
            HDimConst('Unit','clients in new treatment'),
            HDimConst('Period', '2018-03-31T00:00:00/P1Y'),
            HDimConst('Basis of treatment','injecting status')
            ]

In [4]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
if is_interactive():
    savepreviewhtml(c1)

In [5]:
new_table = c1.topandas()

In [6]:
import numpy as np
new_table['OBS'].replace('', np.nan, inplace=True)
new_table.dropna(subset=['OBS'], inplace=True)
new_table.rename(columns={'OBS': 'Value'}, inplace=True)
new_table['Value'] = new_table['Value'].astype(int)
new_table['Value'] = new_table['Value'].map(lambda x:''
                                  if (x == '-')
                                  else int(x))

In [7]:
new_table['Clients in treatment'] = new_table['Clients in treatment'].map(
    lambda x: {
        'Total' : 'All Clients' 
        }.get(x, x))

In [8]:
new_table['Treatment group'] = new_table['Treatment group'].map(
    lambda x: {
        'Total' : 'All' 
        }.get(x, x))

In [9]:
def user_perc(x):
    
    if x == None:
        return 'All inclusice Inconsistent/missing'
    else:
        return x
    
new_table['Treatment group'] = new_table.apply(lambda row: user_perc(row['Treatment group']), axis = 1)

In [10]:
new_table = new_table[['Period','Basis of treatment','Treatment group','Clients in treatment','Measure Type','Value','Unit']]

In [11]:
new_table.head()

,Period,Basis of treatment,Treatment group,Clients in treatment,Measure Type,Value,Unit
0,2018-03-31T00:00:00/P1Y,injecting status,Never injected,Opiate,Count,16165,clients in new treatment
1,2018-03-31T00:00:00/P1Y,injecting status,Never injected,Non-opiate only,Count,14289,clients in new treatment
2,2018-03-31T00:00:00/P1Y,injecting status,Never injected,Non-opiate and Alcohol,Count,16263,clients in new treatment
3,2018-03-31T00:00:00/P1Y,injecting status,Never injected,Alcohol only,Count,48501,clients in new treatment
4,2018-03-31T00:00:00/P1Y,injecting status,Never injected,All Clients,Count,95218,clients in new treatment
